In [1]:
import numpy as np

In [4]:
data = np.arange(21)
data

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20])

In [6]:
np.roll(data,1)

array([20,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19])

In [9]:
PAST= 5
FUTURE = 4

In [19]:
np.vstack((data,np.roll(data,1))).T

array([[ 0, 20],
       [ 1,  0],
       [ 2,  1],
       [ 3,  2],
       [ 4,  3],
       [ 5,  4],
       [ 6,  5],
       [ 7,  6],
       [ 8,  7],
       [ 9,  8],
       [10,  9],
       [11, 10],
       [12, 11],
       [13, 12],
       [14, 13],
       [15, 14],
       [16, 15],
       [17, 16],
       [18, 17],
       [19, 18],
       [20, 19]])

In [40]:
length = PAST + FUTURE

In [44]:
seq = data.copy()
for i in range(length-1):
    seq = np.vstack((seq,np.roll(data,i+1)))
seq = np.flip(seq.T,1)
seq = seq[length-1:,:]
seq

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 2,  3,  4,  5,  6,  7,  8,  9, 10],
       [ 3,  4,  5,  6,  7,  8,  9, 10, 11],
       [ 4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 5,  6,  7,  8,  9, 10, 11, 12, 13],
       [ 6,  7,  8,  9, 10, 11, 12, 13, 14],
       [ 7,  8,  9, 10, 11, 12, 13, 14, 15],
       [ 8,  9, 10, 11, 12, 13, 14, 15, 16],
       [ 9, 10, 11, 12, 13, 14, 15, 16, 17],
       [10, 11, 12, 13, 14, 15, 16, 17, 18],
       [11, 12, 13, 14, 15, 16, 17, 18, 19],
       [12, 13, 14, 15, 16, 17, 18, 19, 20]])

In [47]:
X = seq[:,:PAST]
Y = seq[:,PAST:]
X,Y

(array([[ 0,  1,  2,  3,  4],
        [ 1,  2,  3,  4,  5],
        [ 2,  3,  4,  5,  6],
        [ 3,  4,  5,  6,  7],
        [ 4,  5,  6,  7,  8],
        [ 5,  6,  7,  8,  9],
        [ 6,  7,  8,  9, 10],
        [ 7,  8,  9, 10, 11],
        [ 8,  9, 10, 11, 12],
        [ 9, 10, 11, 12, 13],
        [10, 11, 12, 13, 14],
        [11, 12, 13, 14, 15],
        [12, 13, 14, 15, 16]]),
 array([[ 5,  6,  7,  8],
        [ 6,  7,  8,  9],
        [ 7,  8,  9, 10],
        [ 8,  9, 10, 11],
        [ 9, 10, 11, 12],
        [10, 11, 12, 13],
        [11, 12, 13, 14],
        [12, 13, 14, 15],
        [13, 14, 15, 16],
        [14, 15, 16, 17],
        [15, 16, 17, 18],
        [16, 17, 18, 19],
        [17, 18, 19, 20]]))

In [49]:
FUTURE_STEP = 2
Y[:,::FUTURE_STEP]

array([[ 5,  7],
       [ 6,  8],
       [ 7,  9],
       [ 8, 10],
       [ 9, 11],
       [10, 12],
       [11, 13],
       [12, 14],
       [13, 15],
       [14, 16],
       [15, 17],
       [16, 18],
       [17, 19]])

In [184]:
def single_series2seq(series,past=4,future=1,past_step=1, future_step=1):
    # length of sequences 
    length = past + future
    seq = series.copy()
    for i in range(length-1):
        seq = np.vstack((seq,np.roll(series,i+1)))
    seq = np.flip(seq.T,1)
    seq = seq[length-1:,:]
    X = seq[:,:past:past_step]
    Y = seq[:,past::future_step]

    return X,Y
data = np.arange(10)
print(data)
single_series2seq(data,past = 3, future = 4,future_step=2)

[0 1 2 3 4 5 6 7 8 9]


(array([[0, 1, 2],
        [1, 2, 3],
        [2, 3, 4],
        [3, 4, 5]]),
 array([[3, 5],
        [4, 6],
        [5, 7],
        [6, 8]]))

In [136]:
np.arange(20).reshape(2,-1)[0,:].shape

(10,)

In [205]:
%%timeit
def multi_series2seq(mulseries,**kwarg):
    
    n_series = mulseries.shape[1]
    seq = mulseries.copy()
    
    X,Y = single_series2seq(seq[:,0],past = kwarg['past'],future = kwarg['future'],
                           future_step = kwarg['future_step'])

    for i in range(1,n_series):
        newX,newY = single_series2seq(seq[:,i],past = kwarg['past'],future = kwarg['future'],
                           future_step = kwarg['future_step'])
        X = np.column_stack((X,newX))
        Y = np.column_stack((Y,newY))
    return X,Y
data = np.arange(20).reshape(-1,2)
#print(data)
multi_series2seq(mulseries = data,past = 3, future = 4,future_step=1);

222 µs ± 4.59 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
